# Predictions

In [12]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split

In [13]:
worker_order_df = pd.read_csv('cleanworkorder.csv')

In [14]:
job_df_predict = worker_order_df
job_df_predict = shuffle(job_df_predict, random_state=0)
job_df_predict = job_df_predict[0:10000]

X_wo_D = job_df_predict.loc[:, job_df_predict.columns != "Duration"]
X_wo_AP = job_df_predict.loc[:, job_df_predict.columns != "AffectedProduction"]
X_wo_GPL = job_df_predict.loc[:, job_df_predict.columns != "GrossProductionLoss"]
X_wo_DLF = job_df_predict.loc[:, job_df_predict.columns != "DaysFromLastFailure_int"]

Xs = [X_wo_D, X_wo_AP, X_wo_GPL, X_wo_DLF]
y = ["Duration","AffectedProduction","GrossProductionLoss","DaysFromLastFailure_int"]

for i in range(len(Xs)):
    # Normalization
    curr_norm_scaler = MinMaxScaler().fit(Xs[i])
    Xs[i] = curr_norm_scaler.transform(Xs[i])

    # Scaling
    curr_std_scaler = MinMaxScaler().fit(Xs[i])
    Xs[i] = curr_std_scaler.transform(Xs[i])


X_D_train, X_D_test, y_D_train, y_D_test = train_test_split(Xs[0], job_df_predict['Duration'], test_size=0.2, random_state=42)
X_AP_train, X_AP_test, y_AP_train, y_AP_test = train_test_split(Xs[1], job_df_predict['AffectedProduction'], test_size=0.2, random_state=42)
X_GPL_train, X_GPL_test, y_GPL_train, y_GPL_test = train_test_split(Xs[2], job_df_predict['GrossProductionLoss'], test_size=0.2, random_state=42)
X_DLF_train, X_DLF_test, y_DLF_train, y_DLF_test = train_test_split(Xs[3], job_df_predict['DaysFromLastFailure_int'], test_size=0.2, random_state=42)

splits = [[X_D_train, X_D_test, y_D_train, y_D_test], [X_AP_train, X_AP_test, y_AP_train, y_AP_test], [X_GPL_train, X_GPL_test, y_GPL_train, y_GPL_test], [X_DLF_train, X_DLF_test, y_DLF_train, y_DLF_test]]


In [15]:
models = {
    "Linear Regression": LinearRegression(),
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42),
    "Random Forest Regressor 2": RandomForestRegressor(n_estimators=500, random_state=42), 
    "Support Vector Regressor": SVR(),
    "Support Vector Regressor 2": SVR(kernel= 'rbf', C=100, gamma=0.1, epsilon=0.1),
    "Support Vector Regressor 3": SVR(kernel='linear', C=1, epsilon=0.2, max_iter=10000),
    "Support Vector Regressor 4": SVR(kernel='poly', degree=3, C=10, epsilon=0.1),
    "K-Nearest Neighbors Regressor": KNeighborsRegressor(),
}
i = 0
for split in splits:
    print("Predicting: ", y[i])
    print("")
    for name, model in models.items():
        print("Training", name)

        model.fit(split[0], split[2])

        y_pred = model.predict(split[1])

        mse = mean_squared_error(split[3], y_pred)
        mae = mean_absolute_error(split[3], y_pred)
        r2 = r2_score(split[3], y_pred)
        print("MSE:", mse)
        # MAE = 0.5 -> This means that on average, the predicted target values are off by 0.5 units from the true target values.
        print("MAE:", mae)
        print("R2:", r2)
        print("")
    i+=1
    print("")
    print("")
    print("")

Predicting:  Duration

Training Linear Regression
MSE: 189.42219678056188
MAE: 4.409601151250547
R2: 0.00029889470575283905

Training Logistic Regression
MSE: 196.97
MAE: 2.737
R2: -0.03953565134671955

Training Decision Tree Regressor
MSE: 135.892
MAE: 2.962
R2: 0.2828117036461978

Training Random Forest Regressor
MSE: 106.05943994999998
MAE: 2.844455
R2: 0.4402570493481671

Training Random Forest Regressor 2
MSE: 110.98586680799998
MAE: 2.863916
R2: 0.41425716940379487

Training Support Vector Regressor
MSE: 195.73241546852296
MAE: 2.770161181589232
R2: -0.033004132628002836

Training Support Vector Regressor 2
MSE: 195.55568138691004
MAE: 2.776513625506563
R2: -0.03207139475601917

Training Support Vector Regressor 3
MSE: 195.91525621970575
MAE: 2.8471784868360257
R2: -0.03396909926948899

Training Support Vector Regressor 4
MSE: 195.1220599256563
MAE: 2.777691409106238
R2: -0.02978289920764965

Training K-Nearest Neighbors Regressor
MSE: 164.17816
MAE: 3.2456000000000005
R2: 0.1335

In [17]:
forest_param_grid = {
    'n_estimators': [500, 1000],
    # 'max_features': [None, 'sqrt', 'log2'],
    'max_depth': [50, 200],
    # 'min_samples_split': [ 5, 10],
    # 'min_samples_leaf': [2, 4],
    # 'bootstrap': [True, False]
}

forest_model = RandomForestRegressor()

i = 0
for split in splits:
    print("Predicting: ", y[i])
    print("")


    grid_search = GridSearchCV(forest_model, forest_param_grid, cv=5)


    grid_search.fit(split[0], split[2])


    print("Best hyperparameters: ", grid_search.best_params_)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(split[1])
    mse = mean_squared_error(split[3], y_pred)
    mae = mean_absolute_error(split[3], y_pred)
    r2 = r2_score(split[3], y_pred)
    # MAE = 0.5 -> This means that on average, the predicted target values are off by 0.5 units from the true target values.

    print("MSE:", mse)
    print("MAE:", mae)
    print("R2:", r2)
    print("")
    i+=1



Predicting:  Duration

Best hyperparameters:  {'max_depth': 50, 'n_estimators': 1000}
MSE: 109.73216514600001
MAE: 2.8553689999999996
R2: 0.42087374844528136

Predicting:  AffectedProduction

Best hyperparameters:  {'max_depth': 200, 'n_estimators': 1000}
MSE: 1653.209754216
MAE: 5.117716000000001
R2: 0.04056463050272929

Predicting:  GrossProductionLoss

Best hyperparameters:  {'max_depth': 200, 'n_estimators': 1000}
MSE: 434.53569156199995
MAE: 4.86575
R2: 0.1209459222979904

Predicting:  DaysFromLastFailure_int

Best hyperparameters:  {'max_depth': 50, 'n_estimators': 1000}
MSE: 287957.54010536586
MAE: 324.4777097440476
R2: 0.31093758924064374



In [ ]:
# param_grid = {
#     'hidden_layer_sizes': [(50, 25), (100, 50), (100,)],
#     'activation': ['relu', 'logistic', 'tanh'],
#     'solver': ['adam', 'sgd'],
#     'alpha': [0.0001, 0.001, 0.01],
#     'batch_size': [32, 64, 128],
#     'learning_rate': ['constant', 'adaptive'],
#     'max_iter': [2500, 5000],
#     'early_stopping': [True, False]
# }

param_grid = {
    'hidden_layer_sizes': [(100, 50)],
    'activation': ['relu', 'logistic', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'batch_size': [128],
    'learning_rate': ['adaptive'],
    'max_iter': [5000, 10000],
    'early_stopping': [True]
}


model = MLPRegressor()


grid_search = GridSearchCV(model, param_grid, cv=5)


grid_search.fit(X_train, y_train)


print("Best hyperparameters: ", grid_search.best_params_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# MAE = 0.5 -> This means that on average, the predicted target values are off by 0.5 units from the true target values.

print("MSE:", mse)
print("MAE:", mae)
print("R2:", r2)

